In [1]:
from google.colab import drive
drive.mount('/content/drive')

ERROR: Error in parse(text = x, srcfile = src): <text>:1:6: unexpected symbol
1: from google.colab
         ^


In [2]:
rm(list = ls())


library(dplyr)
library(tidyr)
library(ez)
library(emmeans)
library(ggplot2)
library(rcompanion)
if (!require(rcompanion)) {
  install.packages("rcompanion")
  library(rcompanion)
}


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




ERROR: Error in library(ez): there is no package called ‘ez’


In [3]:
ls

function (name, pos = -1L, envir = as.environment(pos), all.names = FALSE, 
    pattern, sorted = TRUE) 
{
    if (!missing(name)) {
        pos <- tryCatch(name, error = function(e) e)
        if (inherits(pos, "error")) {
            name <- substitute(name)
            if (!is.character(name)) 
                name <- deparse(name)
            warning(gettextf("%s converted to character string", 
                sQuote(name)), domain = NA)
            pos <- name
        }
    }
    all.names <- .Internal(ls(envir, all.names, sorted))
    if (!missing(pattern)) {
        if ((ll <- length(grep("[", pattern, fixed = TRUE))) && 
            ll != length(grep("]", pattern, fixed = TRUE))) {
            if (pattern == "[") {
                pattern <- "\\["
                warning("replaced regular expression pattern '[' by  '\\\\['")
            }
            else if (length(grep("[^\\\\]\\[<-", pattern))) {
                pattern <- sub("\\[<-", "\\\\\\[<-", pattern)
                warning("replaced '[<-' by '\\\\[<-' in regular expression pattern")
            }
        }
        grep(pattern, all.names, value = TRUE)
    }
    else all.names
}
<bytecode: 0x5c9895cfa610>
<environment: namespace:base>

In [ ]:
# Load the original (AUC_orig) and lesion (AUC_lesion) AUC data
AUC_orig <- read.csv("D:/Toronto/stero_EEG/stats/sim_AUC_original.csv")
AUC_lesion <- read.csv("D:/Toronto/stero_EEG/stats/sim_AUC_lesion.csv")
stimulated_network <- read.csv('D:/Toronto/stero_EEG/stats/stimulated_network_combined.csv')



# Define the number of sessions for each participant
sessions_per_participant <- c(7, 13, 9, 7, 9, 8, 10, 11, 2, 4, 7, 17, 3, 8, 16, 8, 10,
                              10, 6, 9, 17, 11, 9, 13, 5, 10, 5, 4, 11, 10, 3, 8, 5, 12,
                              13, 13)

# Create data frames for sessions_per_participant for orig and lesion data
session_data_orig <- data.frame(
  Participant = rep(1:36, times = sessions_per_participant)
)

session_data_lesion <- data.frame(
  Participant = rep(1:36, times = sessions_per_participant)
)

# Combine the orig AUC data, lesion AUC data, and session data
combined_data_orig <- cbind(AUC_orig, session_data_orig)
combined_data_lesion <- cbind(AUC_lesion, session_data_lesion)

# Add a column to indicate the data source (orig or lesion)
combined_data_orig$Data_Source <- "orig"
combined_data_lesion$Data_Source <- "lesion"

# Combine the two data sets vertically to create the final combined_data variable
combined_data <- rbind(combined_data_orig, combined_data_lesion)

# Rename columns if needed
colnames(combined_data) <- c("Early", "Medium", "Late", "Participant", "condition")


combined_data$stimulated_network <- stimulated_network$stimulated_network


# Convert "Early," "Medium," and "Late" to numeric
combined_data$Early <- as.numeric(combined_data$Early)
combined_data$Medium <- as.numeric(combined_data$Medium)
combined_data$Late <- as.numeric(combined_data$Late)

combined_data_long <- combined_data %>%
  pivot_longer(cols = c("Early", "Medium", "Late"), names_to = "Time", values_to = "Value")



model <- aov(Value ~ condition * Time * stimulated_network + Error(Participant/(condition * Time * stimulated_network)), data = combined_data_long)
summary(model)



# Create a vector of unique network levels
network_levels <- unique(combined_data$stimulated_network)

# Select the time point and network of interest
time_point <- "Late"
network_of_interest <- "Default" #"Vis", "SomMot", "DorsAttn", "SalVentAttn", "Limbic", "Cont", "Default")


# Filter the data for the selected time point and network
subset_data <- combined_data_long %>%
  filter(Time == time_point, stimulated_network == network_of_interest)

# Create an empty data frame to store the results
results_df <- data.frame(Network = character(0), U_Statistic = numeric(0), p_Value = numeric(0))

# Set the number of permutations (e.g., 1000 permutations)
num_permutations <- 1000

# Loop through all networks
for (network in network_levels) {
  # Filter the data for the current network
  subset_data_network <- subset_data %>%
    filter(stimulated_network == network)

  # Ensure that there are enough data points for the statistical test
  if (nrow(subset_data_network) >= 2) {
    # Compute the observed test statistic (Mann-Whitney U statistic)
    obs_statistic <- wilcox.test(Value ~ condition, data = subset_data_network, paired = FALSE)

    # Initialize an empty vector to store permuted test statistics
    permuted_statistics <- numeric(num_permutations)

    # Perform permutation testing
    for (k in 1:num_permutations) {
      # Shuffle the labels within the subsetted data for all conditions
      permuted_data <- subset_data_network
      permuted_data$condition <- sample(permuted_data$condition)

      # Compute the test statistic for the permuted data
      perm_statistic <- wilcox.test(Value ~ condition, data = permuted_data, paired = FALSE)

      # Store the permuted test statistic
      permuted_statistics[k] <- perm_statistic$statistic
    }

    # Calculate the p-value by comparing the observed statistic to the permuted distribution
    p_value <- mean(permuted_statistics >= obs_statistic$statistic)

    # Print diagnostic information
    print(paste("Network:", network))
    print(obs_statistic)
    print(p_value)

    # Add the results to the data frame
    results_df <- rbind(results_df, data.frame(Network = network, U_Statistic = obs_statistic$statistic, p_Value = p_value))
  }
}

# Print the results data frame
print(results_df)